Connecting to IBKR on port 7497
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2106
Error 0: 2106
Error 0: 2106
Error 0: 2158

Found 3 vertical spreads:
                  Position Title        Low Strike Leg  Low Strike Position  Low Strike Market Value       High Strike Leg  High Strike Position  High Strike Market Value  Total Market Value Spread Type Expiry Date
GOOGL Jun 27'25 145/155 Bear Put GOOGL Jun 27'25 145 P                    1                   4680.8 GOOGL Jun 27'25 155 P                    -1                 -10718.93            -6038.13    Bear Put  2025-06-27
    WMT Jun 27'25 84/94 Bear Put    WMT Jun 27'25 84 P                    1                   5766.8    WMT Jun 27'25 94 P                    -1                  -6732.93             -966.13    Bear Put  2025-06-27
 TSLA Jun 27'25 275/285 Bear Put  TSLA Jun 27'25 275 P                    1                  52063.8  TSLA Jun 27'25 285 P                    -1                 -74435.93           -22372.13    Bear

In [1]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import queue
from datetime import datetime
from collections import defaultdict

# Configuration
PAPER_TRADING_PORT = 7497
LIVE_TRADING_PORT = 7496
USE_LIVE_TRADING = False  # Set to True for live trading

class PositionWrapper(EWrapper):
    """Wrapper class for IB API callbacks"""
    def __init__(self):
        EWrapper.__init__(self)
        self.positions = []
        self.positions_received = False
        self.data_queue = queue.Queue()
        self.next_req_id = 1000  # Start higher to avoid conflicts
        self.market_data = {}
        self.req_id_to_position = {}  # Maps request IDs to position indices
        self.market_data_received = set()
        
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        print(f"Error {reqId}: {errorCode} - {errorString}")
        # Mark failed requests
        if reqId in self.req_id_to_position and errorCode in [321, 200, 354, 10090]:
            self.market_data_received.add(reqId)
        self.data_queue.put(("error", (reqId, errorCode, errorString)))
        
    def position(self, account, contract, position, avgCost):
        if contract.secType == 'OPT' and position != 0:
            self.positions.append({
                'symbol': contract.symbol,
                'expiry': contract.lastTradeDateOrContractMonth,
                'strike': contract.strike,
                'right': contract.right,
                'position': position,
                'avgCost': avgCost,
                'contract': contract
            })
        
    def positionEnd(self):
        self.positions_received = True
        self.data_queue.put(("positions_complete", None))
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Handle market data price updates"""
        if reqId not in self.market_data:
            self.market_data[reqId] = {}
        
        # Store price based on tick type
        # 1 = bid, 2 = ask, 4 = last, 9 = close
        self.market_data[reqId][tickType] = price
        
        # Mark this request as having received data
        self.market_data_received.add(reqId)
        self.data_queue.put(("tick_price", (reqId, tickType, price)))
    
    def tickSize(self, reqId, tickType, size):
        """Handle market data size updates"""
        pass
    
    def getNextRequestId(self):
        """Get the next available request ID"""
        req_id = self.next_req_id
        self.next_req_id += 1
        return req_id

class PositionClient(EClient):
    """Client class to connect to IB API"""
    def __init__(self, wrapper):
        EClient.__init__(self, wrapper)

class PositionApp:
    """Main application class for getting positions and market data"""
    def __init__(self, host='127.0.0.1', port=7497):
        self.host = host
        self.port = port
        self.wrapper = PositionWrapper()
        self.client = PositionClient(self.wrapper)
        self.connected = False
        self.api_thread = None
    
    def _run_client(self):
        """Run the client in a separate thread"""
        try:
            self.client.run()
        except Exception as e:
            print(f"Error in client thread: {str(e)}")
    
    def connect(self):
        """Connect to IBKR"""
        print(f"Connecting to IBKR at {self.host}:{self.port}")
        
        try:
            self.client.connect(self.host, self.port, clientId=2)  # Use different client ID
        except Exception as e:
            print(f"Connection error: {str(e)}")
            return False
        
        # Start API thread
        self.api_thread = threading.Thread(target=self._run_client, daemon=True)
        self.api_thread.start()
        
        # Wait for connection
        time.sleep(2)
        
        # Set market data type to delayed
        self.client.reqMarketDataType(3)  # 3 = delayed data
        
        self.connected = True
        return True
    
    def disconnect(self):
        """Disconnect from IBKR"""
        if self.connected:
            print("Disconnecting from IBKR")
            try:
                self.client.disconnect()
                self.connected = False
            except Exception as e:
                print(f"Error disconnecting: {str(e)}")
    
    def create_option_contract(self, symbol, expiry, strike, right):
        """Create an option contract"""
        contract = Contract()
        contract.symbol = symbol
        contract.secType = "OPT"
        contract.exchange = "SMART"
        contract.currency = "USD"
        contract.lastTradeDateOrContractMonth = expiry
        contract.strike = strike
        contract.right = right
        return contract
    
    def get_current_price(self, market_data):
        """Get the best available current price from market data"""
        if not market_data:
            return None
        
        # Priority: 1) Last price, 2) Mid-point of bid/ask, 3) Close price, 4) Individual bid/ask
        last = market_data.get(4)  # Last price
        bid = market_data.get(1)   # Bid
        ask = market_data.get(2)   # Ask
        close = market_data.get(9) # Close
        
        # If we have last price and it's valid
        if last is not None and last > 0:
            return last
        
        # If we have both bid and ask, use mid-point
        if bid is not None and ask is not None and bid > 0 and ask > 0:
            return (bid + ask) / 2
        
        # Fall back to close price
        if close is not None and close > 0:
            return close
        
        # If we only have bid or ask, use that
        if ask is not None and ask > 0:
            return ask
        if bid is not None and bid > 0:
            return bid
        
        return None
    
    def get_positions_with_market_data(self):
        """Get positions and fetch current market data"""
        if not self.connected:
            return []
        
        # Request positions
        self.client.reqPositions()
        
        # Wait for positions
        timeout = 10
        start = time.time()
        while not self.wrapper.positions_received and (time.time() - start) < timeout:
            try:
                msg_type, msg_data = self.wrapper.data_queue.get(timeout=0.1)
                if msg_type == "positions_complete":
                    break
            except queue.Empty:
                continue
        
        if not self.wrapper.positions_received:
            print("Timeout waiting for positions")
            return []
        
        print(f"Found {len(self.wrapper.positions)} option positions")
        
        # Request market data for each position
        print("Requesting market data for positions...")
        
        for i, pos in enumerate(self.wrapper.positions):
            req_id = self.wrapper.getNextRequestId()
            self.wrapper.req_id_to_position[req_id] = i
            
            # Create contract for market data request
            contract = self.create_option_contract(
                pos['symbol'], pos['expiry'], pos['strike'], pos['right']
            )
            
            print(f"Requesting data for {pos['symbol']} {pos['strike']} {pos['right']} (reqId: {req_id})")
            
            try:
                # Request market data snapshot
                self.client.reqMktData(req_id, contract, "", True, False, [])
                time.sleep(0.2)  # Small delay between requests
            except Exception as e:
                print(f"Error requesting market data for reqId {req_id}: {e}")
                self.wrapper.market_data_received.add(req_id)
        
        # Wait for market data
        print("Waiting for market data...")
        timeout = 15
        start = time.time()
        expected_requests = len(self.wrapper.req_id_to_position)
        
        while len(self.wrapper.market_data_received) < expected_requests and (time.time() - start) < timeout:
            try:
                msg_type, msg_data = self.wrapper.data_queue.get(timeout=0.5)
                if msg_type == "tick_price":
                    req_id, tick_type, price = msg_data
                    pos_idx = self.wrapper.req_id_to_position.get(req_id)
                    if pos_idx is not None:
                        pos = self.wrapper.positions[pos_idx]
                        print(f"Received price for {pos['symbol']} {pos['strike']}: ${price} (type: {tick_type})")
            except queue.Empty:
                continue
        
        print(f"Received market data for {len(self.wrapper.market_data_received)}/{expected_requests} positions")
        
        # Cancel all market data requests
        for req_id in self.wrapper.req_id_to_position.keys():
            try:
                self.client.cancelMktData(req_id)
            except:
                pass
        
        # Add market data to positions
        for req_id, pos_idx in self.wrapper.req_id_to_position.items():
            if pos_idx < len(self.wrapper.positions):
                market_data = self.wrapper.market_data.get(req_id, {})
                self.wrapper.positions[pos_idx]['market_data'] = market_data
                
                # Debug: Show market data received
                if market_data:
                    pos = self.wrapper.positions[pos_idx]
                    print(f"Market data for {pos['symbol']} {pos['strike']}: {market_data}")
        
        return self.wrapper.positions

def get_vertical_spreads():
    # Connect to IBKR
    port = LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT
    app = PositionApp(port=port)
    
    if not app.connect():
        print("Failed to connect to IBKR")
        return pd.DataFrame()
    
    try:
        # Get positions with market data
        positions = app.get_positions_with_market_data()
        
        if not positions:
            print("No option positions found")
            return pd.DataFrame()
        
        # Process positions into spreads
        spreads = []
        grouped = defaultdict(list)
        
        # Group by symbol, expiry, and option type
        for pos in positions:
            key = (pos['symbol'], pos['expiry'], pos['right'])
            grouped[key].append(pos)
        
        # Find vertical spreads
        for (symbol, expiry, right), position_group in grouped.items():
            if len(position_group) >= 2:
                position_group.sort(key=lambda x: x['strike'])
                
                for i in range(len(position_group)):
                    for j in range(i+1, len(position_group)):
                        pos1, pos2 = position_group[i], position_group[j]
                        
                        # Check for opposite positions (vertical spread)
                        if (pos1['position'] > 0) != (pos2['position'] > 0):
                            spread_type = get_spread_type(pos1, pos2, right)
                            if spread_type:
                                spreads.append(create_spread_row(pos1, pos2, spread_type, app))
        
        return pd.DataFrame(spreads)
    
    finally:
        app.disconnect()

def get_spread_type(pos1, pos2, right):
    """Determine spread type - pos1 is lower strike"""
    if right == 'P':  # Puts
        if pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bull Put'
        elif pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bear Put'
    elif right == 'C':  # Calls
        if pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bull Call'
        elif pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bear Call'
    return None

def create_spread_row(pos1, pos2, spread_type, app):
    """Create DataFrame row for the spread with current market values and opening costs"""
    symbol = pos1['symbol']
    expiry = format_date(pos1['expiry'])
    low_strike = int(min(pos1['strike'], pos2['strike']))
    high_strike = int(max(pos1['strike'], pos2['strike']))
    
    # Determine which position is which strike
    if pos1['strike'] < pos2['strike']:
        low_pos, high_pos = pos1, pos2
    else:
        low_pos, high_pos = pos2, pos1
    
    title = f"{symbol} {expiry} {low_strike}/{high_strike} {spread_type}"
    low_leg = f"{symbol} {expiry} {low_strike} {pos1['right']}"
    high_leg = f"{symbol} {expiry} {high_strike} {pos1['right']}"
    
    # Calculate opening costs (avgCost is already the full contract value)
    low_strike_opening_cost = float(low_pos['avgCost']) * float(low_pos['position'])
    high_strike_opening_cost = float(high_pos['avgCost']) * float(high_pos['position'])
    total_opening_cost = low_strike_opening_cost + high_strike_opening_cost
    
    # Get current market prices
    low_current_price = app.get_current_price(low_pos.get('market_data', {}))
    high_current_price = app.get_current_price(high_pos.get('market_data', {}))
    
    # Calculate current market values
    if low_current_price is not None:
        low_strike_market_value = low_current_price * float(low_pos['position'])*100
    else:
        low_strike_market_value = low_strike_opening_cost  # Fall back to opening cost
        
    if high_current_price is not None:
        high_strike_market_value = high_current_price * float(high_pos['position'])*100
    else:
        high_strike_market_value = high_strike_opening_cost  # Fall back to opening cost
    
    total_market_value = low_strike_market_value + high_strike_market_value
    
    return {
        'Position Title': title,
        'Low Strike Leg': low_leg,
        'Low Strike Position': int(low_pos['position']),
        'Low Strike Opening Cost': round(low_strike_opening_cost, 2),
        'Low Strike Market Value': round(low_strike_market_value, 2),
        'High Strike Leg': high_leg,
        'High Strike Position': int(high_pos['position']),
        'High Strike Opening Cost': round(high_strike_opening_cost, 2),
        'High Strike Market Value': round(high_strike_market_value, 2),
        'Total Opening Cost': round(total_opening_cost, 2),
        'Total Market Value': round(total_market_value, 2),
        'P&L': round(total_market_value - total_opening_cost, 2),
        'Spread Type': spread_type,
        'Expiry Date': datetime.strptime(pos1['expiry'], '%Y%m%d').date()
    }

def format_date(expiry_str):
    """Convert YYYYMMDD to 'Mon DD'YY' format"""
    dt = datetime.strptime(expiry_str, '%Y%m%d')
    return dt.strftime("%b %d'%y")

if __name__ == "__main__":
    print(f"Connecting to IBKR on port {LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT}")
    
    print("\nNote: Using the same market data approach as your working price_monitor system")
    print("- Delayed market data (type 3)")
    print("- Proper threading and queue handling") 
    print("- Fallback to opening costs if market data unavailable")
    
    df = get_vertical_spreads()
    
    if not df.empty:
        print(f"\nFound {len(df)} vertical spreads:")
        print(df.to_string(index=False))
        
        # Optional: Display with better formatting
        print("\n" + "="*100)
        print("DETAILED BREAKDOWN:")
        print("="*100)
        
        for _, row in df.iterrows():
            print(f"\n{row['Position Title']}")
            print(f"  Low Strike:  {row['Low Strike Leg']} | Position: {row['Low Strike Position']}")
            print(f"    Opening Cost: ${row['Low Strike Opening Cost']:,.2f} | Current Market Value: ${row['Low Strike Market Value']:,.2f}")
            print(f"  High Strike: {row['High Strike Leg']} | Position: {row['High Strike Position']}")
            print(f"    Opening Cost: ${row['High Strike Opening Cost']:,.2f} | Current Market Value: ${row['High Strike Market Value']:,.2f}")
            print(f"  TOTALS - Opening Cost: ${row['Total Opening Cost']:,.2f} | Market Value: ${row['Total Market Value']:,.2f} | P&L: ${row['P&L']:,.2f}")
            print(f"  Expiry: {row['Expiry Date']}")
    else:
        print("No vertical spreads found")

/tmp/ipykernel_9545/1047011252.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Connecting to IBKR on port 7497

Note: Using the same market data approach as your working price_monitor system
- Delayed market data (type 3)
- Proper threading and queue handling
- Fallback to opening costs if market data unavailable
Connecting to IBKR at 127.0.0.1:7497
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2104
Error -1: 0 - 2106
Error -1: 0 - 2106
Error -1: 0 - 2106
Error -1: 0 - 2158
Found 8 option positions
Requesting market data for positions...
Requesting data for V 340.0 P (reqId: 1000)
Requesting data for WMT 84.0 P (reqId: 1001)
Requesting data for TSLA 275.0 P (reqId: 1002)
Requesting data for WMT 94.0 P (reqId: 1003)
Requesting data for GOOGL 145.0 P (reqId: 1004)
Requesting data for GOOGL 155.0 P (reqId: 1005)
Requesting data for V 330.0 P (reqId: 1006)
Requesting data for TSLA 285.0 P (reqId: 1007)
Waiting for market data...
Received market data for 8/8 positions
Market data for V 340.0: {1: -1.0, 2: -1.0}
Market data for WMT 84.0: {1: -1.0, 2: -1.0}
Market

In [3]:
df

,Position Title,Low Strike Leg,Low Strike Position,Low Strike Opening Cost,Low Strike Market Value,High Strike Leg,High Strike Position,High Strike Opening Cost,High Strike Market Value,Total Opening Cost,Total Market Value,P&L,Spread Type,Expiry Date
0,GOOGL Jun 27'25 145/155 Bear Put,GOOGL Jun 27'25 145 P,1,46.81,14.0,GOOGL Jun 27'25 155 P,-1,-107.19,-26.0,-60.38,-12.0,48.38,Bear Put,2025-06-27
1,WMT Jun 27'25 84/94 Bear Put,WMT Jun 27'25 84 P,1,57.67,13.0,WMT Jun 27'25 94 P,-1,-67.33,-120.0,-9.66,-107.0,-97.34,Bear Put,2025-06-27
2,TSLA Jun 27'25 275/285 Bear Put,TSLA Jun 27'25 275 P,1,520.64,255.0,TSLA Jun 27'25 285 P,-1,-744.36,-392.0,-223.72,-137.0,86.72,Bear Put,2025-06-27


In [5]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.execution import ExecutionFilter
import threading
import time
from datetime import datetime, timedelta
from collections import defaultdict

# Configuration
PAPER_TRADING_PORT = 7497
LIVE_TRADING_PORT = 7496
USE_LIVE_TRADING = False  # Set to True for live trading

class PositionApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = []
        self.positions_received = False
        self.executions = []
        self.executions_received = False
        self.req_id_counter = 1000
        
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        print(f"Error {errorCode}: {errorString}")
        
    def position(self, account, contract, position, avgCost):
        if contract.secType == 'OPT' and position != 0:
            self.positions.append({
                'symbol': contract.symbol,
                'expiry': contract.lastTradeDateOrContractMonth,
                'strike': contract.strike,
                'right': contract.right,
                'position': position,
                'avgCost': avgCost,
                'localSymbol': contract.localSymbol,
                'conId': contract.conId  # Add contract ID for better matching
            })
        
    def positionEnd(self):
        self.positions_received = True
        
    def execDetails(self, reqId, contract, execution):
        """Receives execution details"""
        if contract.secType == 'OPT':
            self.executions.append({
                'symbol': contract.symbol,
                'localSymbol': contract.localSymbol,
                'conId': contract.conId,
                'strike': contract.strike,
                'right': contract.right,
                'expiry': contract.lastTradeDateOrContractMonth,
                'time': execution.time,
                'side': execution.side,
                'shares': execution.shares,
                'execId': execution.execId
            })
            
    def execDetailsEnd(self, reqId):
        """Called when all executions have been received"""
        self.executions_received = True

def get_vertical_spreads():
    # Connect to IBKR
    port = LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT
    app = PositionApp()
    app.connect("127.0.0.1", port, clientId=1)
    
    # Start thread
    thread = threading.Thread(target=app.run)
    thread.daemon = True
    thread.start()
    time.sleep(1)
    
    # Request positions
    app.reqPositions()
    
    # Wait for positions
    timeout = 10
    start = time.time()
    while not app.positions_received and (time.time() - start) < timeout:
        time.sleep(0.1)
    
    # Request executions for the last 90 days (extended from 30)
    exec_filter = ExecutionFilter()
    exec_filter.time = (datetime.now() - timedelta(days=90)).strftime("%Y%m%d %H:%M:%S")
    
    app.executions_received = False
    app.reqExecutions(app.req_id_counter, exec_filter)
    app.req_id_counter += 1
    
    # Wait for executions
    start = time.time()
    while not app.executions_received and (time.time() - start) < timeout:
        time.sleep(0.1)
    
    app.disconnect()
    
    # DEBUG: Print execution information
    print(f"\nDEBUG: Found {len(app.executions)} total executions")
    if app.executions:
        print("Sample executions:")
        for i, exec_detail in enumerate(app.executions[:3]):
            print(f"  {i+1}. {exec_detail['symbol']} {exec_detail['localSymbol']} - {exec_detail['time']} - {exec_detail['side']}")
    
    print(f"\nDEBUG: Found {len(app.positions)} positions")
    if app.positions:
        print("Sample positions:")
        for i, pos in enumerate(app.positions[:3]):
            print(f"  {i+1}. {pos['symbol']} {pos['localSymbol']} - Position: {pos['position']}")
    
    # Process positions into spreads
    spreads = []
    grouped = defaultdict(list)
    
    # Group by symbol, expiry, and option type
    for pos in app.positions:
        key = (pos['symbol'], pos['expiry'], pos['right'])
        grouped[key].append(pos)
    
    # Create multiple execution lookup strategies
    execution_dates_by_local = defaultdict(list)
    execution_dates_by_conid = defaultdict(list)
    execution_dates_by_details = defaultdict(list)
    
    for exec_detail in app.executions:
        # Strategy 1: By local symbol
        execution_dates_by_local[exec_detail['localSymbol']].append(exec_detail)
        
        # Strategy 2: By contract ID
        execution_dates_by_conid[exec_detail['conId']].append(exec_detail)
        
        # Strategy 3: By symbol + strike + right + expiry
        key = (exec_detail['symbol'], exec_detail['strike'], exec_detail['right'], exec_detail['expiry'])
        execution_dates_by_details[key].append(exec_detail)
    
    # Find vertical spreads
    for (symbol, expiry, right), positions in grouped.items():
        if len(positions) >= 2:
            positions.sort(key=lambda x: x['strike'])
            
            for i in range(len(positions)):
                for j in range(i+1, len(positions)):
                    pos1, pos2 = positions[i], positions[j]
                    
                    # Check for opposite positions (vertical spread)
                    if (pos1['position'] > 0) != (pos2['position'] > 0):
                        spread_type = get_spread_type(pos1, pos2, right)
                        if spread_type:
                            # Try multiple strategies to find opening dates
                            opening_date1 = get_opening_date_multi_strategy(
                                pos1, execution_dates_by_local, execution_dates_by_conid, execution_dates_by_details
                            )
                            opening_date2 = get_opening_date_multi_strategy(
                                pos2, execution_dates_by_local, execution_dates_by_conid, execution_dates_by_details
                            )
                            
                            # Use the later date as spread opening
                            spread_opening_date = max(opening_date1, opening_date2) if opening_date1 and opening_date2 else (opening_date1 or opening_date2)
                            
                            spreads.append(create_spread_row(pos1, pos2, spread_type, spread_opening_date))
    
    return pd.DataFrame(spreads)

def get_opening_date_multi_strategy(position, exec_by_local, exec_by_conid, exec_by_details):
    """Try multiple strategies to find the opening date"""
    
    # Strategy 1: Match by local symbol
    opening_date = get_opening_date_from_executions(position, exec_by_local.get(position['localSymbol'], []))
    if opening_date:
        return opening_date
    
    # Strategy 2: Match by contract ID
    opening_date = get_opening_date_from_executions(position, exec_by_conid.get(position['conId'], []))
    if opening_date:
        return opening_date
    
    # Strategy 3: Match by contract details
    key = (position['symbol'], position['strike'], position['right'], position['expiry'])
    opening_date = get_opening_date_from_executions(position, exec_by_details.get(key, []))
    if opening_date:
        return opening_date
    
    return None

def get_opening_date_from_executions(position, executions):
    """Find the opening date for a position from a list of executions"""
    if not executions:
        return None
    
    # Sort executions by time
    executions.sort(key=lambda x: x['time'])
    
    # Find the first execution that opened the position
    position_sign = 1 if position['position'] > 0 else -1
    
    for exec_detail in executions:
        exec_sign = 1 if exec_detail['side'] == 'BOT' else -1
        if exec_sign == position_sign:
            # Found the opening execution
            try:
                # Try different time formats
                for time_format in ["%Y%m%d  %H:%M:%S", "%Y%m%d %H:%M:%S", "%Y%m%d-%H:%M:%S"]:
                    try:
                        return datetime.strptime(exec_detail['time'], time_format).date()
                    except ValueError:
                        continue
            except:
                pass
    
    return None

def get_spread_type(pos1, pos2, right):
    """Determine spread type - pos1 is lower strike"""
    if right == 'P':  # Puts
        if pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bull Put'
        elif pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bear Put'
    elif right == 'C':  # Calls
        if pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bull Call'
        elif pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bear Call'
    return None

def create_spread_row(pos1, pos2, spread_type, opening_date=None):
    """Create DataFrame row for the spread"""
    symbol = pos1['symbol']
    expiry = format_date(pos1['expiry'])
    low_strike = int(min(pos1['strike'], pos2['strike']))
    high_strike = int(max(pos1['strike'], pos2['strike']))
    
    # Determine which position is which strike
    if pos1['strike'] < pos2['strike']:
        low_pos, high_pos = pos1, pos2
    else:
        low_pos, high_pos = pos2, pos1
    
    title = f"{symbol} {expiry} {low_strike}/{high_strike} {spread_type}"
    low_leg = f"{symbol} {expiry} {low_strike} {pos1['right']}"
    high_leg = f"{symbol} {expiry} {high_strike} {pos1['right']}"
    
    # Calculate market value (simplified)
    market_value = (float(low_pos['avgCost']) * float(low_pos['position']) + 
                   float(high_pos['avgCost']) * float(high_pos['position'])) * 100
    
    return {
        'Position Title': title,
        'Low Strike Leg': low_leg,
        'Low Strike Position': int(low_pos['position']),
        'High Strike Leg': high_leg,
        'High Strike Position': int(high_pos['position']),
        'Market Value': round(market_value, 2),
        'Spread Type': spread_type,
        'Expiry Date': datetime.strptime(pos1['expiry'], '%Y%m%d').date(),
        'Opening Date': opening_date
    }

def format_date(expiry_str):
    """Convert YYYYMMDD to 'Mon DD'YY' format"""
    dt = datetime.strptime(expiry_str, '%Y%m%d')
    return dt.strftime("%b %d'%y")

if __name__ == "__main__":
    print(f"Connecting to IBKR on port {LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT}")
    
    df = get_vertical_spreads()
    
    if not df.empty:
        print(f"\nFound {len(df)} vertical spreads:")
        print(df.to_string(index=False))
        
        # Show opening date analysis
        if 'Opening Date' in df.columns:
            print("\n" + "="*50)
            print("OPENING DATE ANALYSIS")
            print("="*50)
            
            found_dates = 0
            for index, row in df.iterrows():
                opening_date = row['Opening Date']
                if opening_date:
                    days_held = (datetime.now().date() - opening_date).days
                    print(f"\n{row['Position Title']}:")
                    print(f"  Opening Date: {opening_date}")
                    print(f"  Days Held: {days_held}")
                    print(f"  Market Value: ${row['Market Value']:,.2f}")
                    found_dates += 1
                else:
                    print(f"\n{row['Position Title']}: Opening date not found")
            
            if found_dates == 0:
                print(f"\nNo opening dates found. This could mean:")
                print("1. Positions were opened more than 90 days ago")
                print("2. Execution data format mismatch")
                print("3. Different account used for opening vs current positions")
    else:
        print("No vertical spreads found")

Connecting to IBKR on port 7497
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2106
Error 0: 2106
Error 0: 2106
Error 0: 2158
Error 0: 2174

DEBUG: Found 0 total executions

DEBUG: Found 8 positions
Sample positions:
  1. V V     250711P00340000 - Position: -1
  2. WMT WMT   250627P00084000 - Position: 1
  3. TSLA TSLA  250627P00275000 - Position: 1

Found 4 vertical spreads:
                  Position Title        Low Strike Leg  Low Strike Position       High Strike Leg  High Strike Position  Market Value Spread Type Expiry Date Opening Date
    V Jul 11'25 330/340 Bear Put     V Jul 11'25 330 P                    1     V Jul 11'25 340 P                    -1      -9787.63    Bear Put  2025-07-11         None
    WMT Jun 27'25 84/94 Bear Put    WMT Jun 27'25 84 P                    1    WMT Jun 27'25 94 P                    -1       -966.13    Bear Put  2025-06-27         None
 TSLA Jun 27'25 275/285 Bear Put  TSLA Jun 27'25 275 P                    1  TSLA Jun 27'25 285 P       

In [4]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
import threading
import time
from datetime import datetime, timedelta
from collections import defaultdict

# Configuration
PAPER_TRADING_PORT = 7497
LIVE_TRADING_PORT = 7496
USE_LIVE_TRADING = False

class PositionApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = []
        self.positions_received = False
        
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        # Only print non-informational errors
        if errorCode not in [2104, 2106, 2158, 2174]:
            print(f"Error {errorCode}: {errorString}")
        
    def position(self, account, contract, position, avgCost):
        if contract.secType == 'OPT' and position != 0:
            self.positions.append({
                'symbol': contract.symbol,
                'expiry': contract.lastTradeDateOrContractMonth,
                'strike': contract.strike,
                'right': contract.right,
                'position': position,
                'avgCost': avgCost,
                'localSymbol': contract.localSymbol
            })
        
    def positionEnd(self):
        self.positions_received = True

def get_vertical_spreads_dataframe():
    """Get vertical spreads and return as pandas DataFrame"""
    
    # Connect to IBKR
    port = LIVE_TRADING_PORT if USE_LIVE_TRADING else PAPER_TRADING_PORT
    app = PositionApp()
    app.connect("127.0.0.1", port, clientId=1)
    
    # Start thread
    thread = threading.Thread(target=app.run)
    thread.daemon = True
    thread.start()
    time.sleep(1)
    
    # Request positions
    app.reqPositions()
    
    # Wait for positions
    timeout = 10
    start = time.time()
    while not app.positions_received and (time.time() - start) < timeout:
        time.sleep(0.1)
    
    app.disconnect()
    
    # Process positions into spreads
    spreads_data = []
    grouped = defaultdict(list)
    
    # Group by symbol, expiry, and option type
    for pos in app.positions:
        key = (pos['symbol'], pos['expiry'], pos['right'])
        grouped[key].append(pos)
    
    # Find vertical spreads
    for (symbol, expiry, right), positions in grouped.items():
        if len(positions) >= 2:
            positions.sort(key=lambda x: x['strike'])
            
            for i in range(len(positions)):
                for j in range(i+1, len(positions)):
                    pos1, pos2 = positions[i], positions[j]
                    
                    # Check for opposite positions (vertical spread)
                    if (pos1['position'] > 0) != (pos2['position'] > 0):
                        spread_type = get_spread_type(pos1, pos2, right)
                        if spread_type:
                            spread_data = create_spread_data(pos1, pos2, spread_type)
                            spreads_data.append(spread_data)
    
    # Create DataFrame
    if spreads_data:
        df = pd.DataFrame(spreads_data)
        # Add estimated opening dates
        df = add_estimated_opening_dates(df)
        return df
    else:
        # Return empty DataFrame with correct columns
        columns = [
            'Position_Title', 'Symbol', 'Expiry_Date', 'Spread_Type',
            'Low_Strike', 'Low_Strike_Position', 'High_Strike', 'High_Strike_Position',
            'Width', 'Market_Value', 'Days_To_Expiry', 'Opening_Date_Estimated', 
            'Days_Held_Estimated', 'Daily_PnL_Estimate'
        ]
        return pd.DataFrame(columns=columns)

def get_spread_type(pos1, pos2, right):
    """Determine spread type - pos1 is lower strike"""
    if right == 'P':  # Puts
        if pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bull Put'
        elif pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bear Put'
    elif right == 'C':  # Calls
        if pos1['position'] > 0 and pos2['position'] < 0:
            return 'Bull Call'
        elif pos1['position'] < 0 and pos2['position'] > 0:
            return 'Bear Call'
    return None

def create_spread_data(pos1, pos2, spread_type):
    """Create dictionary with spread data"""
    symbol = pos1['symbol']
    expiry_date = datetime.strptime(pos1['expiry'], '%Y%m%d').date()
    expiry_formatted = expiry_date.strftime("%b %d'%y")
    
    # Determine strikes
    low_strike = int(min(pos1['strike'], pos2['strike']))
    high_strike = int(max(pos1['strike'], pos2['strike']))
    width = high_strike - low_strike
    
    # Determine which position is which strike
    if pos1['strike'] < pos2['strike']:
        low_pos, high_pos = pos1, pos2
    else:
        low_pos, high_pos = pos2, pos1
    
    # Calculate market value
    market_value = (float(low_pos['avgCost']) * float(low_pos['position']) + 
                   float(high_pos['avgCost']) * float(high_pos['position'])) * 100
    
    # Days to expiry
    days_to_expiry = (expiry_date - datetime.now().date()).days
    
    return {
        'Position_Title': f"{symbol} {expiry_formatted} {low_strike}/{high_strike} {spread_type}",
        'Symbol': symbol,
        'Expiry_Date': expiry_date,
        'Spread_Type': spread_type,
        'Low_Strike': low_strike,
        'Low_Strike_Position': int(low_pos['position']),
        'High_Strike': high_strike,
        'High_Strike_Position': int(high_pos['position']),
        'Width': width,
        'Market_Value': round(market_value, 2),
        'Days_To_Expiry': days_to_expiry
    }

def add_estimated_opening_dates(df):
    """Add estimated opening dates and related calculations"""
    current_date = datetime.now().date()
    
    estimated_opening_dates = []
    days_held_list = []
    daily_pnl_list = []
    
    for _, row in df.iterrows():
        expiry_date = row['Expiry_Date']
        days_to_expiry = row['Days_To_Expiry']
        spread_type = row['Spread_Type']
        market_value = row['Market_Value']
        
        # Estimate opening date based on spread type and typical holding periods
        if 'Bear' in spread_type:
            # Bear spreads often opened 30-45 days before expiry
            typical_hold_days = 35
        else:
            # Bull spreads often held for 25-40 days
            typical_hold_days = 30
        
        # Adjust based on current days to expiry
        if days_to_expiry > 0:
            # Position still active
            estimated_hold_days = min(typical_hold_days, abs(days_to_expiry - typical_hold_days) + 5)
        else:
            # Position expired
            estimated_hold_days = typical_hold_days
        
        estimated_opening_date = current_date - timedelta(days=estimated_hold_days)
        
        # Calculate daily P&L estimate
        if estimated_hold_days > 0:
            daily_pnl = market_value / estimated_hold_days
        else:
            daily_pnl = 0
        
        estimated_opening_dates.append(estimated_opening_date)
        days_held_list.append(estimated_hold_days)
        daily_pnl_list.append(round(daily_pnl, 2))
    
    df['Opening_Date_Estimated'] = estimated_opening_dates
    df['Days_Held_Estimated'] = days_held_list
    df['Daily_PnL_Estimate'] = daily_pnl_list
    
    return df

def display_spreads_summary(df):
    """Display a nice summary of the spreads"""
    if df.empty:
        print("No vertical spreads found.")
        return
    
    print(f"VERTICAL SPREADS SUMMARY ({len(df)} positions)")
    print("=" * 80)
    
    # Summary statistics
    total_market_value = df['Market_Value'].sum()
    avg_days_held = df['Days_Held_Estimated'].mean()
    total_daily_pnl = df['Daily_PnL_Estimate'].sum()
    
    print(f"Total Market Value: ${total_market_value:,.2f}")
    print(f"Average Days Held: {avg_days_held:.1f}")
    print(f"Estimated Daily P&L: ${total_daily_pnl:,.2f}")
    print()
    
    # Individual positions
    for _, row in df.iterrows():
        print(f"{row['Position_Title']}")
        print(f"   Market Value: ${row['Market_Value']:,}")
        print(f"   Estimated Opening: {row['Opening_Date_Estimated']}")
        print(f"   Days Held: {row['Days_Held_Estimated']}")
        print(f"   Days to Expiry: {row['Days_To_Expiry']}")
        print(f"   Daily P&L: ${row['Daily_PnL_Estimate']:,.2f}")
        print()

# Main execution
if __name__ == "__main__":
    print("Connecting to IBKR and fetching positions...")
    
    # Get the DataFrame
    spreads_df = get_vertical_spreads_dataframe()
    
    # Display summary
    display_spreads_summary(spreads_df)
    
    # Show the full DataFrame
    if not spreads_df.empty:
        print("\nDETAILED DATAFRAME:")
        print("=" * 80)
        print(spreads_df.to_string(index=False))
        
        # Show DataFrame info
        print(f"\nDataFrame Shape: {spreads_df.shape}")
        print(f"Columns: {list(spreads_df.columns)}")
    
    # The DataFrame is now available as 'spreads_df' for further analysis
    print(f"\nDataFrame 'spreads_df' is ready for analysis!")

Connecting to IBKR and fetching positions...
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2106
Error 0: 2106
Error 0: 2106
Error 0: 2158
VERTICAL SPREADS SUMMARY (4 positions)
Total Market Value: $-39,164.02
Average Days Held: 29.5
Estimated Daily P&L: $-1,405.33

V Jul 11'25 330/340 Bear Put
   Market Value: $-9,787.63
   Estimated Opening: 2025-06-01
   Days Held: 19
   Days to Expiry: 21
   Daily P&L: $-515.14

WMT Jun 27'25 84/94 Bear Put
   Market Value: $-966.13
   Estimated Opening: 2025-05-18
   Days Held: 33
   Days to Expiry: 7
   Daily P&L: $-29.28

TSLA Jun 27'25 275/285 Bear Put
   Market Value: $-22,372.13
   Estimated Opening: 2025-05-18
   Days Held: 33
   Days to Expiry: 7
   Daily P&L: $-677.94

GOOGL Jun 27'25 145/155 Bear Put
   Market Value: $-6,038.13
   Estimated Opening: 2025-05-18
   Days Held: 33
   Days to Expiry: 7
   Daily P&L: $-182.97


DETAILED DATAFRAME:
                  Position_Title Symbol Expiry_Date 

NameError: name 'date1' is not defined

In [5]:
import pandas as pd
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.execution import ExecutionFilter
import threading
import time
from datetime import datetime, timedelta
from collections import defaultdict

class PositionApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.positions = []
        self.positions_received = False
        self.executions = []
        self.executions_received = False
        self.commission_reports = []
        
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        if errorCode not in [2104, 2106, 2158, 2174]:
            print(f"Error {errorCode}: {errorString}")
        
    def position(self, account, contract, position, avgCost):
        if contract.secType == 'OPT' and position != 0:
            self.positions.append({
                'symbol': contract.symbol,
                'expiry': contract.lastTradeDateOrContractMonth,
                'strike': contract.strike,
                'right': contract.right,
                'position': position,
                'avgCost': avgCost,
                'localSymbol': contract.localSymbol,
                'conId': contract.conId
            })
        
    def positionEnd(self):
        self.positions_received = True
        
    def execDetails(self, reqId, contract, execution):
        print(f"DEBUG: Found execution - {contract.symbol} {execution.time} {execution.side} {execution.shares}")
        self.executions.append({
            'symbol': contract.symbol,
            'localSymbol': contract.localSymbol,
            'conId': contract.conId,
            'time': execution.time,
            'side': execution.side,
            'shares': execution.shares,
            'price': execution.price,
            'orderId': execution.orderId,
            'execId': execution.execId
        })
        
    def execDetailsEnd(self, reqId):
        print(f"DEBUG: Execution details end. Total executions: {len(self.executions)}")
        self.executions_received = True
        
    def commissionReport(self, commissionReport):
        self.commission_reports.append(commissionReport)

def get_actual_opening_dates():
    """Get actual opening dates from IBKR execution history"""
    
    port = 7497 if not USE_LIVE_TRADING else 7496
    app = PositionApp()
    app.connect("127.0.0.1", port, clientId=1)
    
    thread = threading.Thread(target=app.run)
    thread.daemon = True
    thread.start()
    time.sleep(2)
    
    # Get positions first
    print("Requesting positions...")
    app.reqPositions()
    
    timeout = 15
    start = time.time()
    while not app.positions_received and (time.time() - start) < timeout:
        time.sleep(0.1)
    
    print(f"Found {len(app.positions)} option positions")
    
    # Try multiple execution filter approaches
    print("Requesting execution history...")
    
    # Approach 1: No filter (all executions)
    exec_filter1 = ExecutionFilter()
    app.reqExecutions(1001, exec_filter1)
    
    time.sleep(3)
    
    # Approach 2: Last 365 days
    exec_filter2 = ExecutionFilter()
    exec_filter2.time = (datetime.now() - timedelta(days=365)).strftime("%Y%m%d %H:%M:%S")
    app.reqExecutions(1002, exec_filter2)
    
    time.sleep(3)
    
    # Approach 3: Only options
    exec_filter3 = ExecutionFilter()
    exec_filter3.secType = "OPT"
    app.reqExecutions(1003, exec_filter3)
    
    time.sleep(3)
    
    # Approach 4: By symbol
    symbols = list(set(pos['symbol'] for pos in app.positions))
    for i, symbol in enumerate(symbols):
        exec_filter = ExecutionFilter()
        exec_filter.symbol = symbol
        exec_filter.secType = "OPT"
        app.reqExecutions(1100 + i, exec_filter)
        time.sleep(1)
    
    print("Waiting for execution data...")
    time.sleep(5)
    
    app.disconnect()
    
    print(f"FINAL: Found {len(app.executions)} total executions")
    
    if app.executions:
        print("Sample executions:")
        for exec_detail in app.executions[:5]:
            print(f"  {exec_detail}")
    
    return app.positions, app.executions

# Configuration
USE_LIVE_TRADING = False

if __name__ == "__main__":
    positions, executions = get_actual_opening_dates()

Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2104
Error 0: 2106
Error 0: 2106
Error 0: 2106
Error 0: 2158
Requesting positions...
Found 8 option positions
Requesting execution history...
DEBUG: Execution details end. Total executions: 0
Error 0: 2174
DEBUG: Execution details end. Total executions: 0
DEBUG: Execution details end. Total executions: 0
DEBUG: Execution details end. Total executions: 0
DEBUG: Execution details end. Total executions: 0
DEBUG: Execution details end. Total executions: 0
DEBUG: Execution details end. Total executions: 0
Waiting for execution data...
FINAL: Found 0 total executions
